# Ejercicio 1
* Dados dos vectores A y B, calcular su producto escalar

In [ ]:
import numpy as np
import time

N = 10_000
timing = {}
rng = np.random.default_rng(seed=42)

A = rng.random((N))
B = rng.random((N))

t1 = time.time()
dot_product = 0
for i in range(N):
    dot_product += A[i] * B[i]
t2 = time.time()
loop = t2-t1
timing['loop'] = loop
print("Elapsed ", t2-t1)

In [ ]:
t1 = time.time()

dot_product = np.dot(A, B)

t2 = time.time()

numpy = t2-t1

timing['numpy'] = numpy
print("Elapsed ", t2-t1)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.figure(figsize=(10,6))
plt.title("Measure acceleration of looping versus Numpy dot_prodcut [Lower is better]",fontsize=12)
plt.ylabel("Time in seconds",fontsize=12)
plt.xlabel("Various types of operations",fontsize=14)
plt.grid(True)
plt.bar(x = list(timing.keys()), height= list(timing.values()), align='center',tick_label=list(timing.keys()))

# Ejercicio 2
* En finanzas matemáticas, la ecuación de Black-Scholes es una ecuación diferencial parcial (PDE) que rige la evolución del precio de una opción europea de compra o venta europea según el modelo de [Black-Scholes](https://en.wikipedia.org/wiki/Black%E2%80%93Scholes_equation)

* Una fórmula de negociación de opciones financieras utilizada para el precio de inversión estimados
* La fórmula calcula el precio de las opciones europeas 'put' y 'call'

###  Version bucle
* La función *black_scholes* se implementa con un bucle que calcula las opciones 'put' y 'call'
* Sustituir dicha tipología en forma de bucle por llamadas a la librería **NumPy** para acelerar dicha función

In [ ]:
import numpy.random as rnd

# make xrange available in python 3
try:
    xrange
except NameError:
    xrange = range

SEED = 7777777
S0L = 10.0
S0H = 50.0
XL = 10.0
XH = 50.0
TL = 1.0
TH = 2.0
RISK_FREE = 0.1
VOLATILITY = 0.2
TEST_ARRAY_LENGTH = 1024

###############################################

def gen_data(nopt):
    return (
        rnd.uniform(S0L, S0H, nopt),
        rnd.uniform(XL, XH, nopt),
        rnd.uniform(TL, TH, nopt),
        )

from math import log, sqrt, exp, erf
import numpy as np
invsqrt = lambda x: 1.0/sqrt(x)

def black_scholes ( nopt, price, strike, t, rate, vol, call, put ):
    mr = -rate
    sig_sig_two = vol * vol * 2

    for i in range(nopt):
        P = float( price [i] )
        S = strike [i]
        T = t [i]

        a = log(P / S)
        b = T * mr

        z = T * sig_sig_two
        c = 0.25 * z
        y = invsqrt(z)

        w1 = (a - b + c) * y
        w2 = (a - b - c) * y

        d1 = 0.5 + 0.5 * erf(w1)
        d2 = 0.5 + 0.5 * erf(w2)

        Se = exp(b) * S

        call [i] = P * d1 - Se * d2
        put [i] = call [i] - P + Se


In [ ]:
import time

timing = {}

nopt=1_000_000

price, strike, t = gen_data(nopt)
call = [0.0 for i in range(nopt)]
put = [-1.0 for i in range(nopt)]
price=list(price)
strike=list(strike)
t=list(t)


t1=time.time()
black_scholes(nopt, price, strike, t, RISK_FREE, VOLATILITY, call, put)
t2 = time.time()
print("With for loop and appending it took {} seconds".format(t2-t1))
timing['loop'] = (t2-t1)


###  Version NumPy
* Implementar la función *black_scholes_numpy* que calcule las opciones 'put' y 'call' pero utilizando notación numpy para acelerar el 
* La invocación de la función **erf** se realizar haciendo uso de la función [scipy.special.erf](https://docs.scipy.org/doc/scipy/reference/generated/scipy.special.erf.html) que tiene soporte de datos array de numpy

In [ ]:
import scipy as sc


def black_scholes_numpy ( nopt, price, strike, t, rate, vol, call, put ):
    mr = -rate
    sig_sig_two = vol * vol * 2

    P = price
    S = strike
    T = t

    a = np.log(P / S)
    b = T * mr

    z = T * sig_sig_two
    c = 0.25 * z
    y = np.divide(1, np.sqrt(z))

    w1 = np.multiply(a - b + c,  y)
    w2 = np.multiply(a - b - c, y)

    d1 = 0.5 + 0.5 * sc.special.erf(w1)
    d2 = 0.5 + 0.5 * sc.special.erf(w2)

    Se = np.exp(b) * S
    
    call = P * d1 - Se * d2
    put = call - P + Se
   


In [ ]:
t1=time.time()
black_scholes_numpy(nopt, np.array(price), np.array(strike), np.array(t), RISK_FREE, VOLATILITY, call, put)
t2 = time.time()
print("With for Numpy it took {} seconds".format(t2-t1))
timing['numpy'] = (t2-t1)
print("SpeedUp gain: ", timing['loop']/timing['numpy'])
